In [85]:
import pandas as pd
import glob
import os
import re
import numpy as np
import seaborn as sns
import time
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))  # Agranda el ancho de la ventana.
import warnings
warnings.filterwarnings('ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  
pd.set_option('display.max_colwidth', -1)  
pd.set_option('display.max_rows', 100)  
pd.set_option('display.max_columns', 100) 

## Bases Presupuestos ejecutados

In [86]:
path_to_dir = r"C:\Brian\Industrial Plan I07\Ciencia de Datos\Tp\Presupuestos_Obras\historico_presupuesto_separados_por_cuatrimestres/"

### Con Glob obtendré todos los path a los csv dentro de la carpeta especificada

In [87]:
all_paths = glob.glob(path_to_dir + "*.csv")

In [88]:
# Me fijo que encoding tienen los csv
encodes = []
for path in all_paths:
    with open(path, 'a') as f:
        encodes.append(f.encoding)
pd.Series(encodes).unique()

array(['cp1252'], dtype=object)

In [89]:
list_df_1 = []
list_df_2 = []

trimestres_dict = {
    'primer':  1,
    'segundo': 2,
    'tercer':  3,
    'cuarto':  4
}

for i, path in enumerate(all_paths):
    
    # Del nombre del file obtengo el año y trimestre en cuestión.
    file_name = os.path.basename(path)  # extrae el nombre del file del path.
    trimestre = file_name.split("-")[-2]
    año = re.findall("\d{4}", file_name).pop(0)  # uso regex para encontrar el año.
    
    # A mano definí cuáles csv estaban separados por coma y cuales por punto y coma.
    if i in [16, 20, 22, 23, 24, 25, 26, 27, 28, 29]:
        
    # No todos los archivos tienen éxito al leerlos con 'cp1252' encoding por lo tanto si falla 
    # paso a utilizar 'utf-8-sig' encoding que maneja bien tildes y caracteres.
    
        try:
            df = pd.read_csv(path, encoding='cp1252')
            print(i, 'cp')
        except:
            df = pd.read_csv(path, encoding='utf-8-sig')
            print(i, 'utf-8-sig')
    else:
        try:
            
            df = pd.read_csv(path, sep=';', encoding='cp1252')
            print(i, 'cp')
        except:
            df = pd.read_csv(path, sep=';', encoding='utf-8-sig')
            print(i, 'utf-8-sig')
    
    # Agrego año y trimestre a las bases.
    df['AÑO'] = año
    df['TRIMESTRE'] = trimestres_dict[trimestre]
    
    # Normalización de los nombres de las columnas para que coincidan los datasets:
    df.columns = [x.upper() for x in df.columns]
    
    # se detectó dos estructuras. 
    # Aquellos que empezaban con SECTOR y los que empezaban con CAR
    
    if df.columns[0] == 'SECTOR':
        # No venían con problemas en las columnas así que simplemente las agrego a la lista.
        list_df_1.append(df)
        
    elif df.columns[0] == 'CAR':
        # En cambio esta tira de dataframes tenían problemas y se tuvo que normalizar algunos puntos.
        df.columns = pd.Series(df.columns).str.replace("SANCIÃ³N", "SANCION")
        df.columns = pd.Series(df.columns).str.replace("SANCIÓN", "SANCION")
        df.columns = pd.Series(df.columns).str.replace("INC_", "INCISO_")
        df.columns = pd.Series(df.columns).str.replace("INC$", "INCISO")
        df.columns = pd.Series(df.columns).str.replace("INCISCO", "INCISO")
        df.columns = pd.Series(df.columns).str.replace("ECO _DESC", "ECO_DESC")
        df.columns = pd.Series(df.columns).str.replace("FF", "FTE")
        df.columns = pd.Series(df.columns).str.replace("OGESE", "OG")
        df.columns = pd.Series(df.columns).str.replace("EG_DESC", "OG_DESC")
        df.columns = pd.Series(df.columns).str.replace("SUBJ", "SJUR")
        df.columns = pd.Series(df.columns).str.replace("ACTIVIDAD", "ACT")
        df.columns = pd.Series(df.columns).str.replace("OBRA", "OB")
        df.columns = pd.Series(df.columns).str.replace("^PARC", "PAR")
        df.columns = pd.Series(df.columns).str.replace("SPARC", "SPAR")
        df.columns = pd.Series(df.columns).str.replace("SAR", "SPAR")

        list_df_2.append(df)
    
# Separo en dos dataframes unificados que se seguirán normalizando hasta poder concatenarlos.    
df_1 = pd.concat(list_df_1, axis=0)
df_2 = pd.concat(list_df_2, axis=0)

0 cp
1 utf-8-sig
2 utf-8-sig
3 utf-8-sig
4 cp
5 utf-8-sig
6 utf-8-sig
7 utf-8-sig
8 cp
9 utf-8-sig
10 utf-8-sig
11 utf-8-sig
12 cp
13 cp
14 cp
15 cp
16 utf-8-sig
17 cp
18 cp
19 cp
20 utf-8-sig
21 utf-8-sig
22 utf-8-sig
23 utf-8-sig
24 cp
25 utf-8-sig
26 cp
27 cp
28 utf-8-sig
29 utf-8-sig


In [90]:
df_1 = df_1.drop(['SECTOR','SUBSECTOR','MONEDA'], axis=1)

#### Defino un par de funciones para 

In [91]:
def fix_chars(string):
    dictionary = {
        "Ã±":"ñ", "Ã³": "ó", "Ã‘": "Ñ", "Ãƒâ€™": "Ó", "Ã’": "Ó", "ÃŒ": "Í", "ÃƒÅ’": "Í",
        "Âº": "u","Ãƒâ‚¬": "A", "Ã©": "e", "Ã¡": "a", "Ã­":"í","Ã ":"a","à": "a", "Ã€": "A",
        "á": "a", "é": "e", "í": "i","ó": "o", "ú": "u",
        "Á": "A", "É": "E", "Í": "I", "Ó": "O", "Ú": "U",
        "Ò": "O", "Ã¬": "I", "Ã²": "O", "Ãº": "U", "Â€“": "-", "–": "-",
        "Â°": "da"
    }

    for key in dictionary:
        try:
            if key in string:
                string = string.replace(key, dictionary[key])
        except:
            pass
    return string

In [92]:
def normalize_titles(df_columns):
    
    df_columns = pd.Series(df_columns)

    df_columns = df_columns.str.replace("^CAR$", "CARACTER")
    df_columns = df_columns.str.replace("^CAR_DESC$", "DESC_CARACTER")

    df_columns = df_columns.str.replace("^JUR$", "JURISDICCION")
    df_columns = df_columns.str.replace("^JUR_DESC$", "DESC_JURIS")

    df_columns = df_columns.str.replace("^SJUR_DESC$", "DESC_SJURIS")
    df_columns = df_columns.str.replace("^SJUR$", "SUBJURISDICCION")

    df_columns = df_columns.str.replace("^ENT$", "ENTIDAD")
    df_columns = df_columns.str.replace("^ENT_DESC$", "DESC_ENTIDAD")

    df_columns = df_columns.str.replace("^OG$", "OGESE")
    df_columns = df_columns.str.replace("^OG_DESC$", "DESC_OGESE")

    df_columns = df_columns.str.replace("^UE$", "UNIDAD_EJECUTORA")
    df_columns = df_columns.str.replace("^UE_DESC$", "DESC_UE")

    df_columns = df_columns.str.replace("^PROG$", "PROGRAMA")
    df_columns = df_columns.str.replace("^PROG_DESC$", "DESC_PROGRAMA")

    df_columns = df_columns.str.replace("^SPROG$", "SUBPROGRAMA")
    df_columns = df_columns.str.replace("^SPROG_DESC$", "DESC_SPROGRAMA")

    df_columns = df_columns.str.replace("^PROY$", "PROYECTO")
    df_columns = df_columns.str.replace("^PROY_DESC$", "DESC_PROYECTO")

    df_columns = df_columns.str.replace("^ACT$", "ACTIVIDAD")
    df_columns = df_columns.str.replace("^ACT_DESC$", "DESC_ACTIVIDAD")

    df_columns = df_columns.str.replace("^OB$", "OBRA")
    df_columns = df_columns.str.replace("^OB_DESC$", "DESC_OBRA")

    df_columns = df_columns.str.replace("^FIN$", "FINALIDAD")
    df_columns = df_columns.str.replace("^FIN_DESC$", "DESC_FIN")

    df_columns = df_columns.str.replace("^FUN_DESC$", "DESC_FUN")
    df_columns = df_columns.str.replace("^FUN$", "FUNCION")

    df_columns = df_columns.str.replace("^INCISO_DESC$", "DESC_INCISO")

    df_columns = df_columns.str.replace("^PPAL$", "PRINCIPAL")
    df_columns = df_columns.str.replace("^PPAL_DESC$", "DESC_PRINCIPAL")

    df_columns = df_columns.str.replace("^PAR$", "PARCIAL")
    df_columns = df_columns.str.replace("^PAR_DESC$", "DESC_PARCIAL")
    
    df_columns = df_columns.str.replace("^PARC$", "PARCIAL")
    df_columns = df_columns.str.replace("^PARC_DESC$", "DESC_PARCIAL")

    df_columns = df_columns.str.replace("^SPAR$", "SUBPARCIAL")
    df_columns = df_columns.str.replace("^SPAR_DESC$", "DESC_SUBPARCIAL")
    
    df_columns = df_columns.str.replace("^SPARC$", "SUBPARCIAL")
    df_columns = df_columns.str.replace("^SPARC_DESC$", "DESC_SUBPARCIAL")

    df_columns = df_columns.str.replace("^ECO$", "CLAS_ECONOMICO")
    df_columns = df_columns.str.replace("^ECO_DESC$", "DESC_CLAS_ECO")

    df_columns = df_columns.str.replace("^FTE$", "FUENTE_FIN")
    df_columns = df_columns.str.replace("^FTE_DESC$", "DESC_FUENTE_FIN")
    
    df_columns = df_columns.str.replace("^FF$", "FUENTE_FIN")
    df_columns = df_columns.str.replace("^FF_DESC$", "DESC_FUENTE_FIN")

    df_columns = df_columns.str.replace("^GEO$", "UBICACION_GEOGRAFICA")
    df_columns = df_columns.str.replace("^GEO_DESC$", "DESC_UBIC")

    df_columns = df_columns.str.replace("^GEO$", "UBICACION_GEOGRAFICA")
    df_columns = df_columns.str.replace("^GEO_DESC$", "DESC_UBIC")
    
    df_columns = df_columns.str.replace("^OBRA_DESC$", "DESC_OBRA")
    df_columns = df_columns.str.replace("^OGESE_DESC$", "DESC_OGESE")    
    
    df_columns = df_columns.str.replace("OGESE", "SERVICIO") 
    
    df_columns = df_columns.apply(lambda x: fix_chars(x))
        
    return df_columns

In [93]:
df_2.columns = normalize_titles(df_2.columns)
df_1.columns = normalize_titles(df_1.columns)

In [94]:
presupuesto_ejecutado = pd.concat([df_1, df_2], axis=0).reset_index(drop=True)

In [95]:
def fix_chars_in_columns(df_all):
    description_columns = df_all.columns[["DESC_" in col for col in df_all.columns]]    
    
    start_start = time.time()
    asd = len(description_columns)
    
    for descripcion in description_columns:
        start = time.time()
        df_all[descripcion] = df_all[descripcion].apply(lambda string: fix_chars(string))
        print("total time taken this loop: ", round(time.time() - start, 1), "seg.  Faltan ", asd)
        asd -= 1

    print(f"\nTotal time taken: {(time.time() - start_start)/60} min\n\n")
    
    return df_all

In [96]:
def fix_descripcion_strings(df_all):

    description_columns = df_all.columns[["DESC_" in col for col in df_all.columns]] 

    start = time.time()
    count=len(description_columns)
    for descripcion in description_columns:

        start_start = time.time()

        df_all[descripcion] = df_all[descripcion].str.upper()

        df_all.loc[df_all[df_all[descripcion] == "MIN.HÃ\xa0BITAT Y DES. HUMANO"].index,
                                                                 descripcion] = "MINISTERIO HABITAT Y DESARROLO HUMANO"

        habitat_pattern = '|'.join(['MIN.HÃ\xa0BITAT Y DES. HUMANO', 'MIN.HABITAT Y DES. HUMANO',
                                    'MIN. HABITAT Y DES. HUMANO', "MINISTERIO DE HABITAT Y DES. HUMANO",
                                    "MINISTERIO DE DESARROLLO HUMANO Y HABITAT"])

        moden_pattern = '|'.join(['MINISTERIO DE MODERN. INNOVACION Y TECNOLOGIA', 'MIN.MODERN.INNOVACION Y TECNOLOGIA'])

        df_all[descripcion] = df_all[descripcion].str.replace(habitat_pattern, 'MINISTERIO HABITAT Y DESARROLO HUMANO')

        df_all[descripcion] = df_all[descripcion].str.replace(moden_pattern, 'MINISTERIO DE MODERNIZACION')

        df_all[descripcion] = df_all[descripcion].str.replace('MINISTERIO DE HACIENDA$',
                                                                'MINISTERIO DE HACIENDA Y FINANZAS')

        df_all[descripcion] = df_all[descripcion].str.replace('MIN.DESA.ECON Y PROD',
                                                                'MINISTERIO DE DESARROLLO ECONOMICO')

        economy_pattern = '|'.join(["MINISTERIO DE HACIENDA Y FINANZAS", "MINISTERIO DE ECONOMIA Y FINANZAS"])

        df_all[descripcion] = df_all[descripcion].str.replace(economy_pattern, 'MINISTERIO DE ECONOMIA, HACIENDA Y FINANZAS')

        ceys_pattern = '|'.join(["CONSEJO ECONOMICO Y SOCIAL DE LA CABA",
                                 "CONSEJO ECONOMICO Y SOCIAL DE LA CIUDAD DE BUENOS AIRES",
                                 "CONSEJO ECONOMICO Y SOCIAL DE LA C.A.B.A"])

        df_all[descripcion] = df_all[descripcion].str.replace(ceys_pattern, "CONSEJO ECONOMICO Y SOCIAL")

        dg_pattern = '|'.join(["^DG.","^D.G.","^DG.","^DIR.GRAL.","^D.GRAL.","^DIRECCION.GRAL.","^DIR..GRAL.",
                               "^DIR.G.", "^DIREC.GRAL.", "^D..G."])  

        df_all[descripcion] = df_all[descripcion].str.replace(dg_pattern, 'DIRECCION GENERAL ')

        ss_pattern = '|'.join(["^SUBS\.","^SUBS ","^SUBSEC\.","^SUBSEC ","^SUBSECRET\.","^SUBSECRET ",
                               "^SS.","^SUB\.","^SUB ","^SUBSECR\.","^SUBSECR ", "^SUSEC\."])

        df_all[descripcion] = df_all[descripcion].str.replace(ss_pattern, 'SUBSECRETARIA ')

        df_all[descripcion] = df_all[descripcion].str.replace("^CS.", 'CENTROS SALUD ')
        df_all[descripcion] = df_all[descripcion].str.replace("CTRO.DE SALUD", 'CENTRO DE SALUD ')
        df_all[descripcion] = df_all[descripcion].str.replace("FINANCIAMIENTO INTER$", 'FINANCIAMIENTO INTERNO')
        df_all[descripcion] = df_all[descripcion].str.replace("TRANSFERENCIAS INTER$", 'TRANSFERENCIAS INTERNAS')
        df_all[descripcion] = df_all[descripcion].str.replace("FINANCIAMIENTO EXTER$", 'FINANCIAMIENTO EXTERNO')
        df_all[descripcion] = df_all[descripcion].str.replace("TRANSFERENCIAS AFECT$", 'TRANSFERENCIAS AFECTADAS')
        
        df_all[descripcion] = df_all[descripcion].str.replace("MIN.ESP.PUBLICO E HIG. URBANA$",
                                                              'MINISTERIO DE AMBIENTE Y ESPACIO PUBLICO')


        
        

        df_all[descripcion] = df_all[descripcion].str.replace('  ', ' ')

        print("total time taken this loop: ", round(time.time() - start_start, 1), "seg.  Faltan ", count)
        count -= 1

    print(f"\nTotal time taken: {(time.time() - start)/60} min\n\n")
    
    return df_all

In [97]:
def drop_cve_columns(df_all):
    
    df_all['CARACTER']= df_all["DESC_CARACTER"]
    df_all['JURISDICCION']= df_all["DESC_JURIS"]
    df_all['SUBJURISDICCION']= df_all["DESC_SJURIS"]
    df_all['ENTIDAD']= df_all["DESC_ENTIDAD"]
    df_all['UNIDAD_EJECUTORA']= df_all["DESC_UE"]
    df_all['PROGRAMA']= df_all["DESC_PROGRAMA"]
    df_all['SUBPROGRAMA']= df_all["DESC_SPROGRAMA"]
    df_all['PROYECTO']= df_all["DESC_PROYECTO"]
    df_all['ACTIVIDAD']= df_all["DESC_ACTIVIDAD"]
    df_all['OBRA']= df_all["DESC_OBRA"]
    df_all['FINALIDAD']= df_all["DESC_FIN"]
    df_all['FUNCION']= df_all["DESC_FUN"]
    df_all['INCISO']= df_all["DESC_INCISO"]
    df_all['PRINCIPAL']= df_all["DESC_PRINCIPAL"]
    df_all['PARCIAL']= df_all["DESC_PARCIAL"]
    df_all['SUBPARCIAL']= df_all["DESC_SUBPARCIAL"]
    df_all['CLAS_ECONOMICO']= df_all["DESC_CLAS_ECO"]
    df_all['FUENTE_FIN']= df_all["DESC_FUENTE_FIN"]
    df_all['UBICACION_GEOGRAFICA']= df_all["DESC_UBIC"]
    df_all['SERVICIO']= df_all["DESC_SERVICIO"]


    if "DESC_FIN_FUN" in df_all.columns:
        df_all.loc[df_all['FINALIDAD'].isna(), "FINALIDAD"] = df_all.loc[df_all['FINALIDAD'].isna(), "DESC_FIN_FUN"]  
        df_all.loc[df_all['FUNCION'].isna(), "FUNCION"] = df_all.loc[df_all['FUNCION'].isna(), "DESC_FIN_FUN"]   
        df_all = df_all.drop("DESC_FIN_FUN", axis=1)
        
        df_all.loc[df_all["CARACTER"] == 1, "DESC_CARACTER"] = "ADMINISTRACION CENTRAL"
        df_all.loc[df_all["CARACTER"] == 2, "DESC_CARACTER"] = "ORGANISMOS DESCENTRALIZADOS"
    
    df_all = df_all.drop(['DESC_JURIS', 'DESC_SJURIS', 'DESC_ENTIDAD', 'DESC_UE',
                         'DESC_PROGRAMA', 'DESC_SPROGRAMA', 'DESC_PROYECTO', 'DESC_OBRA',
                         'DESC_INCISO', 'DESC_PRINCIPAL', 'DESC_PARCIAL',
                         'DESC_SUBPARCIAL', 'DESC_CLAS_ECO', 'DESC_FUENTE_FIN', 'DESC_UBIC',
                         'DESC_CARACTER', 'DESC_ACTIVIDAD', 'DESC_FIN',
                         'DESC_FUN', "DESC_SERVICIO",
#                           "DESC_FIN_FUN"
                         ], axis=1)

    return df_all

In [98]:
def replace_str_num_values(df_all, col):
    
    df_all[col] = df_all[col].astype(str)
    
    df_all.loc[:, col] = df_all.loc[:, col].str.replace(",", ".")
    df_all.loc[:, col] = df_all.loc[:, col].str.replace("\...$", '')
    df_all.loc[:, col] = df_all.loc[:, col].str.replace("\.", '')
    df_all.loc[:, col] = df_all.loc[:, col].replace(" ", np.NaN)
    
    df_all[col] = df_all[col].astype(float)
    df_all[col] = df_all[col].round(2)
    
    return df_all

In [99]:
presupuesto_ejecutado = fix_chars_in_columns(presupuesto_ejecutado)
presupuesto_ejecutado = fix_descripcion_strings(presupuesto_ejecutado)

total time taken this loop:  14.0 seg.  Faltan  21
total time taken this loop:  10.2 seg.  Faltan  20
total time taken this loop:  11.0 seg.  Faltan  19
total time taken this loop:  8.5 seg.  Faltan  18
total time taken this loop:  9.1 seg.  Faltan  17
total time taken this loop:  9.7 seg.  Faltan  16
total time taken this loop:  9.0 seg.  Faltan  15
total time taken this loop:  8.1 seg.  Faltan  14
total time taken this loop:  8.0 seg.  Faltan  13
total time taken this loop:  42.4 seg.  Faltan  12
total time taken this loop:  7.7 seg.  Faltan  11
total time taken this loop:  7.7 seg.  Faltan  10
total time taken this loop:  8.1 seg.  Faltan  9
total time taken this loop:  8.0 seg.  Faltan  8
total time taken this loop:  8.0 seg.  Faltan  7
total time taken this loop:  8.4 seg.  Faltan  6
total time taken this loop:  7.9 seg.  Faltan  5
total time taken this loop:  14.7 seg.  Faltan  4
total time taken this loop:  14.2 seg.  Faltan  3
total time taken this loop:  14.3 seg.  Faltan  2
t

In [100]:
presupuesto_ejecutado = drop_cve_columns(presupuesto_ejecutado)

In [101]:
presupuesto_ejecutado = replace_str_num_values(presupuesto_ejecutado, 'DEVENGADO')
presupuesto_ejecutado = replace_str_num_values(presupuesto_ejecutado, 'SANCION')
presupuesto_ejecutado = replace_str_num_values(presupuesto_ejecutado, 'VIGENTE')
presupuesto_ejecutado = replace_str_num_values(presupuesto_ejecutado, 'DEFINITIVO')

In [102]:
presupuesto_ejecutado.columns

Index(['CARACTER', 'JURISDICCION', 'SUBJURISDICCION', 'ENTIDAD', 'SERVICIO',
       'UNIDAD_EJECUTORA', 'PROGRAMA', 'SUBPROGRAMA', 'PROYECTO', 'ACTIVIDAD',
       'OBRA', 'FINALIDAD', 'FUNCION', 'INCISO', 'PRINCIPAL', 'PARCIAL',
       'SUBPARCIAL', 'CLAS_ECONOMICO', 'FUENTE_FIN', 'UBICACION_GEOGRAFICA',
       'VIGENTE', 'DEVENGADO', 'AÑO', 'TRIMESTRE', 'SANCION', 'DEFINITIVO'],
      dtype='object')

In [103]:
presupuesto_ejecutado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247216 entries, 0 to 1247215
Data columns (total 26 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   CARACTER              1049043 non-null  object 
 1   JURISDICCION          1247211 non-null  object 
 2   SUBJURISDICCION       1247211 non-null  object 
 3   ENTIDAD               1247211 non-null  object 
 4   SERVICIO              1247211 non-null  object 
 5   UNIDAD_EJECUTORA      1247180 non-null  object 
 6   PROGRAMA              1247180 non-null  object 
 7   SUBPROGRAMA           1247180 non-null  object 
 8   PROYECTO              1246992 non-null  object 
 9   ACTIVIDAD             1048791 non-null  object 
 10  OBRA                  1246817 non-null  object 
 11  FINALIDAD             1247211 non-null  object 
 12  FUNCION               1247211 non-null  object 
 13  INCISO                1247211 non-null  object 
 14  PRINCIPAL             1247211 non-

In [104]:
presupuesto_ejecutado.to_csv("presupuestos_ejecutados_clean.csv", index=False)

### Bases Presupuestos Sancionados

In [105]:
path_to_dir = r"C:\Brian\Industrial Plan I07\Ciencia de Datos\Tp\Presupuesto_Sancionado/"

In [106]:
all_paths = glob.glob(path_to_dir + "*.csv")

In [107]:
encodes = []
for path in all_paths:
    with open(path, 'a') as f:
        encodes.append(f.encoding)
pd.Series(encodes).unique()

array(['cp1252'], dtype=object)

In [108]:
list_df = []

for file in all_paths:
    
    file_name = os.path.basename(file)
    año = re.findall("\d{4}", file_name).pop(0)
    
    try:
        if file.endswith('2020.csv'):
            df = pd.read_csv(file, sep=',', encoding='cp1252')
            df.columns = pd.Series(df.columns).str.upper()
        else:
            df = pd.read_csv(file, sep=';', encoding='cp1252')
    except:
        if file.endswith('2019.csv'):
            df = pd.read_csv(file, sep=',', encoding='utf-8-sig')
            df.columns = pd.Series(df.columns).str.upper()
        else:
            df = pd.read_csv(file, sep=';', encoding='utf-8-sig')  
            
    finally:
        df['AÑO'] = año
        
        df.columns = pd.Series(df.columns).str.replace("INC_", "INCISO_")
        df.columns = pd.Series(df.columns).str.replace("INC$", "INCISO")
        df.columns = pd.Series(df.columns).str.replace("SPOR_DESC$", "SPROG_DESC")
        df.columns = normalize_titles(df.columns)
        list_df.append(df)
        
presupuesto_sancionado = pd.concat(list_df)

In [109]:
presupuesto_sancionado.shape

(640405, 42)

In [110]:
presupuesto_sancionado = fix_chars_in_columns(presupuesto_sancionado)
presupuesto_sancionado = fix_descripcion_strings(presupuesto_sancionado)

total time taken this loop:  4.5 seg.  Faltan  20
total time taken this loop:  4.5 seg.  Faltan  19
total time taken this loop:  4.4 seg.  Faltan  18
total time taken this loop:  4.4 seg.  Faltan  17
total time taken this loop:  4.1 seg.  Faltan  16
total time taken this loop:  4.0 seg.  Faltan  15
total time taken this loop:  4.1 seg.  Faltan  14
total time taken this loop:  3.9 seg.  Faltan  13
total time taken this loop:  4.0 seg.  Faltan  12
total time taken this loop:  4.1 seg.  Faltan  11
total time taken this loop:  4.2 seg.  Faltan  10
total time taken this loop:  4.1 seg.  Faltan  9
total time taken this loop:  3.8 seg.  Faltan  8
total time taken this loop:  3.9 seg.  Faltan  7
total time taken this loop:  4.0 seg.  Faltan  6
total time taken this loop:  4.5 seg.  Faltan  5
total time taken this loop:  4.4 seg.  Faltan  4
total time taken this loop:  4.0 seg.  Faltan  3
total time taken this loop:  4.5 seg.  Faltan  2
total time taken this loop:  3.9 seg.  Faltan  1

Total ti

In [111]:
presupuesto_sancionado = drop_cve_columns(presupuesto_sancionado)

In [114]:
presupuesto_sancionado = replace_str_num_values(presupuesto_sancionado, 'SANCION')

In [115]:
presupuesto_sancionado.shape

(640405, 22)

In [116]:
presupuesto_sancionado.to_csv("presupuesto_sancionado_clean.csv", index=False)